In [3]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "plotly"
import datetime

In [3]:
###########################################################################################################################

In [4]:
# read inputs

full = pd.read_csv('D:/CEEW/Smart Meter Analysis/Data Acquisition and Preprocessing/Cleaned data 02Mar21_corrected.csv')
survey = pd.read_csv('FINAL- Appliance stock survey and metadata 20Nov19.xlsx - FINAL- Appliance stock survey a.csv')

# knowing the ac households
ac_list = list(survey.loc[survey['q_409_ac_yn']==1]['home_no'])

# Remove the meters with scarce data
ac_list = list(set(ac_list) - set(['BR17','BR20','BR42']))

print('Number of AC households',len(ac_list))

Number of AC households 31


In [ ]:
################### ROUGH

In [4]:
full = pd.read_csv('D:/CEEW/Smart Meter Analysis/GSP_energy_disaggregator/FINAL/Disaggregation/stage2_filter/3 min disaggregated result FINAL with removed meters.csv')

In [5]:
full.drop(['compressor','voltage','m-y','gsp'],axis=1,inplace=True)

In [6]:
full.set_index(pd.to_datetime(full['x_Timestamp']),inplace=True)

In [7]:
start_date = '2019-08-01' # from may
end_date = '2019-08-31'  # till september
mask = (full.index >= start_date) & (full.index < end_date)
df = full.loc[mask] 

In [8]:
df = df.loc[df['meter'].isin(['MH07','MH10','MH14','MH21','BR08','BR23','BR38'])]

In [9]:
df.drop(['x_Timestamp'],axis=1,inplace=True)

In [10]:
df.rename_

,power,current,final_compressor,ac_power,ac_on,meter
x_Timestamp,,,,,,
2019-08-01 00:00:00,1680.0,9.67,1,1392.259743,1,MH07
2019-08-01 00:03:00,1780.0,10.17,1,1392.259743,1,MH07
2019-08-01 00:06:00,1800.0,10.12,1,1392.259743,1,MH07
2019-08-01 00:09:00,1760.0,10.11,1,1392.259743,1,MH07
2019-08-01 00:12:00,1760.0,10.13,1,1392.259743,1,MH07
...,...,...,...,...,...,...
2019-08-30 23:45:00,200.0,1.20,0,880.590938,1,MH21
2019-08-30 23:48:00,1080.0,4.89,0,880.590938,1,MH21
2019-08-30 23:51:00,1520.0,6.60,0,880.590938,1,MH21


In [11]:
df.to_csv('D:/CEEW/Smart Meter Analysis/GSP_energy_disaggregator/Sujitha/CEEW - Disaggregated AC data 22Sept21.csv')

In [ ]:
###########################################################################################################################

In [ ]:
def preprocess(meter):
    df = full.loc[full['meter']==meter]
    df.index = pd.to_datetime(df['x_Timestamp'])
    start_date = '2019-05-01' # from may
    end_date = '2019-11-31'  # till september
    mask = (df.index >= start_date) & (df.index < end_date)
    df1 = df.loc[mask] 
    e1 = df1[['t_kWh']]
    e1 = e1*20000
    df1 = df1[['z_Avg Current (Amp)']]
    start_date = '2020-03-01' # from may
    end_date = '2020-11-31'  # till september
    mask = (df.index >= start_date) & (df.index < end_date)
    df2 = df.loc[mask] 
    e2 = df2[['t_kWh']]
    e2 = e2*20000
    df2 = df2[['z_Avg Current (Amp)']]
#     
    return df1,e1,df2,e2

In [ ]:
##########################################################################################################################

In [ ]:
# GSP

def gsp_(thresh_list, df,meter):
    sigma = thresh_list[0] #20
    ri = thresh_list[1] #0.15
    T_Positive = thresh_list[2];
    T_Negative = thresh_list[3];
    #Following parameters alpha and beta are used in Equation 15 of the paper 
    # alpha define weight given to magnitude and beta define weight given to time
    alpha = thresh_list[4]
    beta  = thresh_list[5]
    win = thresh_list[6]
    # this defines the  minimum number of times an appliance is set ON in considered time duration
    instancelimit = 3

    main_val = df.values # get only readings
    main_ind = df.index  # get only timestamp
    data_vec =  main_val
    # signature_database = "signature_database_labelled.csv" #the signatures were extracted of power analysis from April 28th to 30th
    threshold = 2000 # threshold of DTW algorithm used for appliance power signature matching

    delta_p = [np.round(data_vec[i+1] - data_vec[i], 2) for i in range(0, len(data_vec) - 1)]

    event =  [i for i in range(0, len(delta_p)) if (delta_p[i] > T_Positive or delta_p[i] < T_Negative) ]

    # # initial and refined clustering block of Figure 1 in the paper
    clusters = gsp.refined_clustering_block(event, delta_p, sigma, ri)  #takes time

    # # Feature matching block of Figure 1 in the paper
    finalclusters, pairs = gsp.pair_clusters_appliance_wise(clusters, data_vec, delta_p, instancelimit)
    appliance_pairs = gsp.feature_matching_module(pairs, delta_p, finalclusters, alpha, beta)

    # # create appliance wise disaggregated series
    power_series, appliance_signatures = gsp.generate_appliance_powerseries(appliance_pairs, delta_p)
    power_timeseries = gsp.create_appliance_timeseries(power_series, main_ind)
    gsp_result = pd.concat(power_timeseries, axis = 1)
    
    return gsp_result

In [ ]:
####################################################################################################################################

In [ ]:
#file handling folder names

In [ ]:
####################################################################################################################################

In [ ]:
for meter in ac_list:
    
    df1,e1,df2,e2 = preprocess(meter) 
    thresh_list = [60,0.6,2.9,-2.9,0.5,0.5,200] #current

    gsp_result1 = gsp_(thresh_list, df1, meter)
    gsp_result2 = gsp_(thresh_list, df2, meter)

    gsp_result = pd.concat([gsp_result1, gsp_result2])
    df = pd.concat([df1, df2])
    e = pd.concat([e1,e2])


    gsp_result['Power'] = e
    gsp_result['Current'] = df['z_Avg Current (Amp)']
    gsp_result.to_csv(folder+'_gsp_init'+meter+'.csv')


In [ ]:
####################################################################################################################################